# 🔥 _DeepFire_: API Project for Fire Detection 🔥

In this project, you'll apply your skills at neural network development in a new way: taking a model that you've trained yourself and deploying it to a static webpage that you can work with to upload new images and get prediction accuracy results. 

This project will primarily focus on your abilities in creating and testing neural network architecture development. 

#### **Specifically, you'll be creating a convolutional neural network that can ingest Fire Detection Image Data and predict binary class values, similarly to what we've done with multilayer perceptrons in the past.**

Boilerplate and supporting architectures have been provided for a multitude of tasks ranging from data preprocessing, processing, ingestion, and predictive assessment – however, major tasks and design work will ultimately be left to you to approach and figure out ideal, optimized solutions. 

#### 🔹 General Importations

As always, we'll start with importing basic tools and functions for our task.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import utils

import os, PIL
from glob import glob

import tensorflow as tf

print(tf.__version__)




2.6.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls

Mounted at /content/drive
drive  sample_data


---

#### 🔎 Initializing Deep Learning Tools 🔍

---

> Your first task will be crucial to ensuring the successful implementation of the rest of your notebook. 
> 
> **Initialize each line with the correct function type from the TensorFlow documentation.**
> 
> Feel free to refer throughout the notebook and across previous notebooks to see which TensorFlow architectures you've used for similar tasks. 
> 
> To give you a guide for how this should look, you've been provided with a single correct function declaration in the form of `image_dataset_from_directory` at the end of the cell. 

---

In [ ]:
""" Sequential Model Architecture """
# Initialize the sequential model architecture here.
Sequential = tf.keras.models.Sequential

""" Data Preprocessing Functions """
# Initialize the experimental resizing layer here.
Resizing = tf.keras.layers.experimental.preprocessing.Resizing
# Initialize the experimental rescaling layer here.
Rescaling = tf.keras.layers.experimental.preprocessing.Rescaling
""" Data Augmentation Functions """
# Initialize the experimental random flipping layer here.
RandomFlip = tf.keras.layers.experimental.preprocessing.RandomFlip


# Initialize the experimental random rotating layer here.
RandomRotation = tf.keras.layers.experimental.preprocessing.RandomRotation

# Initialize the experimental random zooming layer here.
RandomZoom = tf.keras.layers.experimental.preprocessing.RandomZoom

""" Artificial Neural Network Layer Inventory """
# Initialize the dense connective layer here.
Dense = tf.keras.layers.Dense
# Initialize the dropout regularization layer here.
Dropout = tf.keras.layers.Dropout

""" Convolutional Neural Network Layer Inventory """
# Initialize the 2D convolutional layer here.
Conv2D = tf.keras.layers.Conv2D
# Initialize the 2D max pooling layer here.
MaxPool2D = tf.keras.layers.MaxPool2D
# Initialize the flattening layer here.
Flatten = tf.keras.layers.Flatten

""" Residual Network Layer Inventory """
# TODO: Initialize the Residual Network multilayer model here.
# TODO: Make sure you initialize the 50-layer residual network! 
# HINT: Look up `ResNet50` for appropriate documentation. 
ResNet50 = tf.keras.applications.ResNet50

""" Function to Load Images from Target Folder """
image_dataset_from_directory = tf.keras.preprocessing.image_dataset_from_directory

In [ ]:
!ls drive/MyDrive/Images



Fire_Images  Normal_Images


#### 🔹 Precheck Image Dataset Sizes

If you've followed instructions carefully from the `project/PROJECT.md` instructions, the following dataset directory instantiations should work perfectly. 

If they do not, double-check to make sure you've saved your dataset to the appropriate location. 

In [ ]:
# Use the `glob.glob` function to show how many images are in each folder
DATA_DIRECTORY = "drive/MyDrive/Images"
FIRE_IMAGES_PATTERN = f"{DATA_DIRECTORY}/Fire_Images/*"
NOT_FIRE_IMAGES_PATTERN = f"{DATA_DIRECTORY}/Normal_Images/*"

print(f"Number of fire image samples: {len(glob(FIRE_IMAGES_PATTERN))}")
print(f"Number of non-fire image samples: {len(glob(NOT_FIRE_IMAGES_PATTERN))}")

Number of fire image samples: 541
Number of non-fire image samples: 541


#### 🔸 Load Dataset

Like we've done previously, let's set our batch size and image dimensions to work seamlessly with our configured model.

In [ ]:
batch_size = 32
IMAGE_HEIGHT = IMAGE_WIDTH = 256

In [ ]:
train = image_dataset_from_directory(
    directory=DATA_DIRECTORY,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=batch_size
)

Found 1082 files belonging to 2 classes.
Using 866 files for training.


In [ ]:
class_names = train.class_names

In [ ]:
validation = image_dataset_from_directory(
    directory=DATA_DIRECTORY,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=batch_size
)

Found 1082 files belonging to 2 classes.
Using 216 files for validation.


From these results, we can actually see that we have a major class imbalance with our fire images representing our minority class. 

Let's go ahead and fix that by resampling our dataset. 

#### 🔹 Resample (Oversample) Minority Class

In [ ]:
def resample_class_distribution(train, DATA_DIRECTORY=DATA_DIRECTORY, save=True):
    """ Helper function to resample class distribution for image dataset. """
    minority_class, majority_class = list(), list()
    for images, labels, in train.take(3):
        for image, label in zip(images, labels):
            if label == 0:
                minority_class.append(image.numpy().astype(np.uint8))
            else:
                majority_class.append(image.numpy().astype(np.uint8))
    FIRE_SIZE = len(glob(f"{DATA_DIRECTORY}/Fire_Images/*"))
    NOT_FIRE_SIZE = len(glob(f"{DATA_DIRECTORY}/Normal_Images/*"))
    upsampled_images = np.array(utils.resample(minority_class, replace=True, 
                                               n_samples=(NOT_FIRE_SIZE - FIRE_SIZE),
                                               random_state=42))
    if save == True:
        index = 0
        for image in upsampled_images:
            PATH = f"{DATA_DIRECTORY}/Fire_Images/new_fire_{index}.png"
            PIL.Image.fromarray(image).save(PATH)
            index += 1

In [ ]:
resample_class_distribution(train)

You should now see that additional images have been generated to balance out both classes prior to predictive modeling.

**Go ahead and re-run the `Load Dataset` steps to see new generated dataset changes.**

#### 🔹 Pre-Optimize Image File Ingestion

This is an accessory step to optimize image data ingestion at the cost of slightly higher memory usage. No modifications are required for this function. 

In [ ]:
def configure_performant_datasets(dataset, shuffling=None):
    """ 
    Custom function to prefetch and cache stored elements
    of retrieved image data to boost latency and performance
    at the cost of higher memory usage. 
    """
    AUTOTUNE = tf.data.AUTOTUNE
    # Cache and prefetch elements of input data for boosted performance
    if not shuffling:
        return dataset.cache().prefetch(buffer_size=AUTOTUNE)
    else:
        return dataset.cache().shuffle(shuffling).prefetch(buffer_size=AUTOTUNE)

In [ ]:
train =         configure_performant_datasets(train, shuffling=1000)
validation =    configure_performant_datasets(validation)

---

#### 🔎 Create Resizing and Normalization Layers 🔍

---

> For this task, you'll declare your resizing and normalization layers using the layer architectures that you imported earlier. 
>
> Recall that for this step, we want to accomplish two key tasks: 
> - Resize all images to the predetermined square image dimensions as indicated by `IMAGE_HEIGHT` and `IMAGE_WIDTH`.
> - Scale all images so pixel values are within the range of (0., 1.) rather than the original (0., 255.).
>
> Additionally, since we're working with colorized image data, we'll want to ensure that our image rescaling/normalization step inputs images as stacks-of-three, since each image channel corresponds to red, green, and blue pixel values. 
>
> As always, refer to previous notebook documentation on image normalization for colorization if you need help.

---

In [ ]:
resizing_layer = Resizing(IMAGE_HEIGHT,IMAGE_WIDTH)

normalization_layer = Rescaling(IMAGE_HEIGHT, IMAGE_WIDTH)

height_factor=(0.2, 0.3)



---

#### 🔎 Neural Network Architecture Creation 🔍

---

> Now time for the main event! 
> 
> Here, you'll be creating and instantiating your model architecture. 
> 
> For this assignment, you'll be creating a **Convolutional Neural Network** that can process Fire Detection images for predictive purposes.
> 
> _For this project, you will not be provided guidance as to how to design and implement your CNN architecture._
> 
> Refer to previous notebooks and challenges on CNNs as well as online documentation/resources for how to design CNN models on higher-order images. 
> 
> **This is a highly creative step, and there are no wrong answers; however, you will be assessed on your experimentation process and why you choose specific modeling layers, configurations, optimizers, regularizers, and overall design choices.**
>
> Light boilerplate will be provided to get you started, but as always, use any and all resources at your disposal to finish the job! 

---

In [ ]:
input_layer = tf.keras.layers.InputLayer(input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3))
flattening_layer = tf.keras.layers.Flatten()
dense_layer_1 = tf.keras.layers.Dense(512, activation="relu")
dense_layer_2 = tf.keras.layers.Dense(256, activation="relu")
dense_layer_3 = tf.keras.layers.Dense(128, activation="relu")
output_layer = tf.keras.layers.Dense(1, activation="sigmoid")
dropout_layer_1 = tf.keras.layers.Dropout(.25)
dropout_layer_2 = tf.keras.layers.Dropout(.25)
dropout_layer_3 = tf.keras.layers.Dropout(.33)
dropout_layer_4 = tf.keras.layers.Dropout(.40)
conv_layer_1 = tf.keras.layers.Conv2D(8, kernel_size=(3, 3),
                               padding="same",
                               activation="swish"
                               )
conv_layer_2 = tf.keras.layers.Conv2D(8,
                               kernel_size=(3, 3),
                               padding="same",
                               activation="swish")
random_flip = tf.keras.layers.RandomFlip("horizontal", input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3))
random_rotation = tf.keras.layers.RandomZoom(height_factor, width_factor=None, fill_mode='reflect', interpolation='bilinear',
    seed=None, name=None)
pooling_layer_1 = tf.keras.layers.MaxPool2D(pool_size=(2, 2))
pooling_layer_2 = tf.keras.layers.MaxPool2D(pool_size=(2, 2))


""" Sequential Model Architecture Setup """
model = Sequential()

""" CNN Layering Steps """
# TODO: Instantiate and Add Layers to Create a Functional CNN for Image Recognition.
# TODO: Be Sure to Use as Many Imported Layers at the Top of the Notebook as Possible.
model.add(input_layer) 
# gonna augment my data here then do everything else
# model.add(random_flip)
# model.add(random_rotation)
model.add(resizing_layer)
model.add(normalization_layer)


model.add(conv_layer_1)
model.add(pooling_layer_1)

# model.add(conv_layer_2)
# model.add(pooling_layer_2)
# model.add(dropout_layer_2)

model.add(flattening_layer)
model.add(dense_layer_1)
model.add(dropout_layer_3)
model.add(dense_layer_2)
model.add(dropout_layer_4)
model.add(dense_layer_3)
model.add(output_layer)

""" CNN Architecture Summarization """
# TODO: Save Screenshots of your Model Summaries to put in this Project Subfolder.
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resizing (Resizing)          (None, 256, 256, 3)       0         
_________________________________________________________________
rescaling (Rescaling)        (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 256, 256, 8)       224       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 8)       0         
_________________________________________________________________
flatten (Flatten)            (None, 131072)            0         
_________________________________________________________________
dense (Dense)                (None, 512)               67109376  
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0

---

#### 🔎 Neural Network Configuration 🔍

---

> For this task, you'll compile your CNN architecture with appropriate parameters for loss calculation, optimization, and accuracy metrics.
> 
> As always, refer to previous notebooks, tutorials, and documentation for best-case parameters to use for image recognition models.

---

In [ ]:
""" CNN Model Compilation """
# TODO: Compile Model with Appropriate Loss, Optimizer, and Metrics-Based Parameters
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

---

#### 🔎 CNN Model Predictive Fitness 🔍

---

> For this task, you'll be taking your compiled model and fitting it against your training and validation data.
> 
> Keep in mind that there are several opportunities for further optimizing your workflow with techniques such as batch normalization, generator-based data feeding, etc. 
> 
> As always, refer to previous notebooks, tutorials, and documentation for designing model fitness with validation data. 

---

In [ ]:
""" CNN Model Fitness and History Extraction """
# TODO: Fit Model Against Training and Validation Data with Appropriate Epochs
epochs = 15
history = model.fit(
    train,
    validation_data= validation,
    epochs=epochs
)

Epoch 1/15


KeyboardInterrupt: ignored

---

#### 🔎 CNN Model Evaluation 🔍

---

> For this task, you'll evaluate your CNN model using the validation dataset in order to calculate overall validation accuracy and loss.
> 
> As always, refer to previous notebooks, tutorials, and documentation for using the proper evaluation function for model prediction. 

---

In [ ]:
""" CNN Model Predictive Evaluation """
# TODO: Evaluate Model Against Validation Dataset
model.evaluate(train, validation)

#### 🔹 Model State Saving

When you are satisfied with your model state configuration and performance and are ready to export the model's weights and parameters for deployment purposes, simply run the following function! 

In [ ]:
def save_model(model, file_name, save_format):
    """ 
    Save the model weights and architecture.
    
    Parameters: 
       model(Model): keras Model object being saved
       file_name(str): name of the Hadoop file where
                       the whole model will be saved
       save_format(str): Indicates whether to save the model to the default
                         SavedModel('tf'), or HDF5('h5'), or 
                         use both H5 and JSON ('composite') formats. 
       Returns: None
    """
    MODEL_DIRECTORY = "../model"
    def __save_as_composite():
      """ Saving the model as H5 (for params) + JSON (for the architecture) """
      # Save the weights
      model.save_weights(f'{MODEL_DIRECTORY}/{file_name}_params.h5')
      # Save the architecture
      with open(f'{MODEL_DIRECTORY}/{file_name}_layers.json', 'w') as f:
          f.write(model.to_json())
    
    def __save_as_h5():
      """ Option 2: Saving whole model as a single H5 file (more storage) """
      model.save(f"{MODEL_DIRECTORY}/{file_name}.h5", save_format=save_format)

    # Call the appropiate save func
    if save_format == 'h5':
      __save_as_h5()
    elif save_format == 'composite':
      __save_as_composite()
    else:  # save as a SavedModel
      model.save(file_name)

In [ ]:
save_model(model, "fire_cnn_classifier", "composite")

Now go ahead and complete the remaining tasks in `project/PROJECT.md` to complete this project successfully! 

---
---
---